In [1]:
input_default_json_conf_file='cnn_emotion4_testing_delay_default.json';

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.image import load_img

2024-05-09 12:52:01.436438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 12:52:01.436460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 12:52:01.437137: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 12:52:01.440859: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 12:52:01.964552: W tensorflow/compiler/tf2

In [3]:
import sys
sys.path.append('../library');

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-05-09 12:52:06.069038: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 12:52:06.090057: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 12:52:06.090163: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [6]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_base_dir     = DATA['dataset_base_dir'];
dataset_labels_file  = DATA['dataset_labels_file'];
dataset_name         = DATA['dataset_name'];

label_colname='label';
filename_colname='filename';
target_labels=['negative','neutro','pain','positive'];

## Model of network
## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type = DATA['model_type'];

## Model filepath
model_dir = DATA['model_dir'];

## times to count time
times = DATA['times'];

## Output
output_base_dir = DATA['output_base_dir'];

##############################################

best_model_file = os.path.join(model_dir,'model_'+model_type+'.h5');


'''
print('   dataset_base_dir:',dataset_base_dir)
print('dataset_labels_file:',dataset_labels_file)
print('       dataset_name:',dataset_name)
print('         model_type:',model_type)
print('          model_dir:',model_dir)
print('              times:',times)
print('    output_base_dir:',output_base_dir)
'''

"\nprint('   dataset_base_dir:',dataset_base_dir)\nprint('dataset_labels_file:',dataset_labels_file)\nprint('       dataset_name:',dataset_name)\nprint('         model_type:',model_type)\nprint('          model_dir:',model_dir)\nprint('              times:',times)\nprint('    output_base_dir:',output_base_dir)\n"

# If command line

In [7]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--dataset-dir':
        dataset_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-file':
        dataset_labels_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    elif sys.argv[n]=='--model-dir':
        model_dir=sys.argv[n+1];
    elif sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];

best_model_file = os.path.join(model_dir,'model_'+model_type+'.h5');

print('   dataset_base_dir:',dataset_base_dir)
print('dataset_labels_file:',dataset_labels_file)
print('       dataset_name:',dataset_name)
print('         model_type:',model_type)
print('          model_dir:',model_dir)
print('              times:',times)
print('    output_base_dir:',output_base_dir)

print('         model_file:',best_model_file)

   dataset_base_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/DATASET/TESE/PATIENT-RECOGNITION/PATIENT-IMAGES/perwi/dataset/test/
dataset_labels_file: labels-emotion4-v1.csv
       dataset_name: perwi
         model_type: efficientnet_b3
          model_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/DOCTORADO2/cnn_emotion4/perwi/training_validation_holdout/efficientnet_b3
              times: 10
    output_base_dir: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/DOCTORADO2/cnn_emotion4_1
         model_file: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/DOCTORADO2/cnn_emotion4/perwi/training_validation_holdout/efficientnet_b3/model_efficientnet_b3.h5


# Set seed of random variables


In [8]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [9]:
# Load filenames and labels
test_data = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(test_data)

                    filename     label
0    patient/filename302.png  Positive
1    patient/filename303.png  Positive
2    patient/filename304.png  Positive
3    patient/filename305.png  Positive
4    patient/filename306.png  Positive
..                       ...       ...
268   people/filename522.png  Positive
269   people/filename525.png    Neutro
270   people/filename526.png  Positive
271   people/filename527.png    Neutro
272   people/filename528.png    Neutro

[273 rows x 2 columns]


# Creating output directory

In [25]:

output_dir = os.path.join(output_base_dir,dataset_name,'test_delay',model_type);

os.makedirs(output_dir,exist_ok = True);

# Create new model

In [26]:
import BodyEmotion4Lib.lib_model as mpp

model, target_size = mpp.create_model(model_type=model_type,load_weights=False,file_of_weight=best_model_file);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Loading architecture efficientnet_b3

        url: https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1
target_size: (300, 300)



2024-05-09 10:10:11.046647: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 10:10:11.046766: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 10:10:11.046847: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loading the weights in: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/DOCTORADO2/cnn_emotion4/perwi/training_validation_holdout/efficientnet_b3/model_efficientnet_b3.h5
Loaded the weights in: /media/fernando/B0EA304AEA300EDA/Dados/Fernando/OUTPUTS/DOCTORADO2/cnn_emotion4/perwi/training_validation_holdout/efficientnet_b3/model_efficientnet_b3.h5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              10783528  
                                                                 
 dense (Dense)               (None, 4)                 6148      
                                                                 
Total params: 10789676 (41.16 MB)
Trainable params: 6148 (24.02 KB)
Non-trainable params: 10783528 (41.14 MB)
_________________________________________________________________


# Creating PIL data images

In [30]:
import pandas as pd

pil_image=[];
label_image=[]
for index, row in test_data.iterrows():
    filepath=os.path.join(dataset_base_dir,row[filename_colname]);
    pil_image.append(load_img(filepath,target_size=target_size));
    label_image.append(target_labels.index(row[label_colname].lower()));

#print(label_image)

# Time

In [ ]:
from pytictoc import TicToc
t = TicToc() 

# Evaluate best model

In [ ]:
Count=0;
L=len(pil_image);

t.tic();
for m in range(times):
    for n in range(L):
        res=mpp.evaluate_model_from_pil(model, pil_image[n],target_size=target_size)
        if label_image[n]==res:
            Count=Count+1;
t0=t.tocvalue();


results = dict()
results['categorical_accuracy']=Count*1.0/(L*times);
results['delayms']=t0*1000.0/(times*L);
print(results,"\n\n");

In [ ]:



with open(os.path.join(output_dir,"results_testing.m"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));


# final all json
with open(os.path.join(output_dir,"testing_data_results.json"), 'w') as f:
    json.dump(results, f,indent=4);

tf.keras.backend.clear_session()
